# Hadoop Streaming assignment 1: Words Rating

Create your own WordCount program and process Wikipedia dump. Use the second job to sort words by quantity in the reverse order (most popular first). Output format:

word <tab> count

The result is the 7th word by popularity and its quantity.

__Hint__: it is possible to use exactly one reducer in the second job to obtain a totally ordered result.

In [1]:
%%writefile word_count_mapper.py

import sys
import re
from collections import Counter

reload(sys)
sys.setdefaultencoding('utf-8')

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    counts = Counter(words)
    for word, count in counts.items():
        print "%s\t%d" % (word.lower(), count)

Writing word_count_mapper.py


In [2]:
%%writefile word_count_reducer.py

import sys

current_key = None
word_sum = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Writing word_count_reducer.py


In [3]:
%%writefile word_rating_mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8')

for line in sys.stdin:
    try:
        word, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    print "%d\t%s" % (count, word)

Writing word_rating_mapper.py


In [4]:
%%writefile word_rating_reducer.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8')

for line in sys.stdin:
    try:
        count, word = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    print "%s\t%d" % (word, count)

Writing word_rating_reducer.py


In [5]:
%%bash

WORD_COUNT_OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files word_count_mapper.py,word_count_reducer.py \
    -mapper "python word_count_mapper.py" \
    -combiner "python word_count_reducer.py" \
    -reducer "python word_count_reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${WORD_COUNT_OUT_DIR} > /dev/null
    
WORD_RATING_OUT_DIR="wordrating_result_"$(date +"%s%6N")
NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${WORD_RATING_OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordRating" \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D stream.map.output.field.separator=\t \
    -D mapreduce.partition.keycomparator.options=-k1,1nr \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files word_rating_mapper.py,word_rating_reducer.py \
    -mapper "python word_rating_mapper.py" \
    -reducer "python word_rating_reducer.py" \
    -input ${WORD_COUNT_OUT_DIR} \
    -output ${WORD_RATING_OUT_DIR} > /dev/null

hdfs dfs -cat ${WORD_RATING_OUT_DIR}/part-00000 | head -7 | tail -1

is	126420


-rm: Not enough arguments: expected 1 but got 0
Usage: hadoop fs [generic options]
	[-appendToFile <localsrc> ... <dst>]
	[-cat [-ignoreCrc] <src> ...]
	[-checksum <src> ...]
	[-chgrp [-R] GROUP PATH...]
	[-chmod [-R] <MODE[,MODE]... | OCTALMODE> PATH...]
	[-chown [-R] [OWNER][:[GROUP]] PATH...]
	[-copyFromLocal [-f] [-p] [-l] [-d] <localsrc> ... <dst>]
	[-copyToLocal [-f] [-p] [-ignoreCrc] [-crc] <src> ... <localdst>]
	[-count [-q] [-h] [-v] [-t [<storage type>]] [-u] [-x] <path> ...]
	[-cp [-f] [-p | -p[topax]] [-d] <src> ... <dst>]
	[-createSnapshot <snapshotDir> [<snapshotName>]]
	[-deleteSnapshot <snapshotDir> <snapshotName>]
	[-df [-h] [<path> ...]]
	[-du [-s] [-h] [-x] <path> ...]
	[-expunge]
	[-find <path> ... <expression> ...]
	[-get [-f] [-p] [-ignoreCrc] [-crc] <src> ... <localdst>]
	[-getfacl [-R] <path>]
	[-getfattr [-R] {-n name | -d} [-e en] <path>]
	[-getmerge [-nl] [-skip-empty-file] <src> <localdst>]
	[-help [cmd ...]]
	[-ls [-C] [-d] [-h] [-q] [-R] [-t] [-S] [-r] [-u